In [1]:
import torch
import torch.nn as nn

In [3]:
GPT_CONFIG_124M = {
 "vocab_size": 50257, # Vocabulary size
 "context_length": 1024, # Context length
 "emb_dim": 768, # Embedding dimension
 "n_heads": 12, # Number of attention heads
 "n_layers": 12, # Number of layers
 "drop_rate": 0.1, # Dropout rate
 "qkv_bias": False # Query-Key-Value bias
}

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length,
                 dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads) == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out)
        self.out_proj = nn.Linear(d_out, d_out)
        self.register_buffer(
            "mask",
            torch.tril(torch.ones(context_length, context_length)))

class LayerNorm(nn.Module):
 def __init__(self, emb_dim):
  super().__init__()
  self.eps = 1e-5
  self.scale = nn.Parameter(torch.ones(emb_dim))
  self.shift = nn.Parameter(torch.zeros(emb_dim))

 def forward(self, x):
  mean = x.mean(dim=-1, keepdim=True)
  var = x.var(dim=-1, keepdim=True, unbiased=False)
  norm_x = (x - mean) / torch.sqrt(var + self.eps)
  return self.scale * norm_x + self.shift

class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(
    nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
    nn.GELU(),
    nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
    )
  def forward(self, x):
    return self.layers(x)

class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(
    d_in=cfg["emb_dim"],
    d_out=cfg["emb_dim"],
    context_length=cfg["context_length"],
    num_heads=cfg["n_heads"],
    dropout=cfg["drop_rate"],
    qkv_bias=cfg["qkv_bias"])
    self.ff = FeedForward(cfg)
    self.norm1 = LayerNorm(cfg["emb_dim"])
    self.norm2 = LayerNorm(cfg["emb_dim"])
    self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

class GPTModel(nn.Module):
 def __init__(self, cfg):
  super().__init__()
  self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
  self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
  self.drop_emb = nn.Dropout(cfg["drop_rate"])

  self.trf_blocks = nn.Sequential(
  *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

  self.final_norm = LayerNorm(cfg["emb_dim"])
  self.out_head = nn.Linear(
  cfg["emb_dim"], cfg["vocab_size"], bias=False
  )
 def forward(self, in_idx):
  batch_size, seq_len = in_idx.shape
  tok_embeds = self.tok_emb(in_idx)

  pos_embeds = self.pos_emb(
  torch.arange(seq_len, device=in_idx.device)
  )
  x = tok_embeds + pos_embeds
  x = self.drop_emb(x)
  x = self.trf_blocks(x)
  x = self.final_norm(x)
  logits = self.out_head(x)
  return logits